# Data was generated based on the Starter Notebook created by John Whitaker (JW)
https://colab.research.google.com/drive/1DPizsNT7GUK776TRDmk5rZVMsB1kJY5H

In [22]:
import tifffile as tiff
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import glob
import datetime
from tqdm.notebook import tqdm

import sklearn

from sklearn.utils.class_weight import compute_class_weight

from sklearn.inspection import permutation_importance
from scipy.stats.mstats import gmean

# Models
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import BaggingClassifier
import catboost
from catboost import CatBoostClassifier

import eli5
from eli5.sklearn import PermutationImportance

from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.metrics import log_loss

import os

# Package versions

In [2]:
catboost.__version__ #0.22

'0.22'

In [4]:
sklearn.__version__ #0.22.2

'0.22.2'

In [23]:
eli5.__version__ #0.10.1

In [6]:
dates_raw = [
datetime.datetime(2019, 6, 6, 0, 0),
datetime.datetime(2019, 7, 1, 0, 0),
datetime.datetime(2019, 7, 6, 0, 0),
datetime.datetime(2019, 7, 11, 0, 0),
datetime.datetime(2019, 7, 21, 0, 0),
datetime.datetime(2019, 8, 5, 0, 0),
datetime.datetime(2019, 8, 15, 0, 0),
datetime.datetime(2019, 8, 25, 0, 0),
datetime.datetime(2019, 9, 9, 0, 0),
datetime.datetime(2019, 9, 19, 0, 0),
datetime.datetime(2019, 9, 24, 0, 0),
datetime.datetime(2019, 10, 4, 0, 0),
datetime.datetime(2019, 11, 3, 0, 0)
]

dates = []

for i in range(13):
    dt = "".join(str(dates_raw[i].date()).split("-"))
    dates.append(dt)

In [7]:
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

In [8]:
# Including Cloud Layer
bands_all = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'CLD']

# DATA GENERATION - based on JW starter code
> NB: Directory of competition data (multi spectral satellite images) should be named "data", and all relevant images should be included in their respective tile folder and sub folders

In [9]:
def load_file(fp):
    """Takes a PosixPath object or string filepath
    and returns np array"""
    
    return tiff.imread(fp.__str__())

In [10]:
row_locs = []
col_locs = []
field_ids = []
labels = []
tiles = []

for tile in range(4):
    fids = f'data/0{tile}/{tile}_field_id.tif'
    labs = f'data/0{tile}/{tile}_label.tif'
    fid_arr = load_file(fids)
    lab_arr = load_file(labs)
    for row in range(len(fid_arr)):
    for col in range(len(fid_arr[0])):
        if fid_arr[row][col] != 0:
            row_locs.append(row)
            col_locs.append(col)
            field_ids.append(fid_arr[row][col])
            labels.append(lab_arr[row][col])
            tiles.append(tile)

In [11]:
df_generated = pd.DataFrame({
    'fid':field_ids,
    'label':labels,
    'row_loc': row_locs,
    'col_loc':col_locs,
    'tile':tiles
})

In [12]:
df_generated.head()

,fid,label,row_loc,col_loc,tile
0,2928,4,214,1278,0
1,2928,4,214,1279,0
2,2928,4,214,1280,0
3,2928,4,214,1281,0
4,2928,4,214,1282,0


In [13]:
df_generated.shape

(67557, 5)

In [14]:
col_names = []
col_values = []

for tile in range(4): # 1) For each tile
    print('Tile: ', tile)
    for d in dates: # 2) For each date
        print(str(d))
#         d = ''.join(str(d.date()).split('-')) # Nice date string
        t = '0' + str(tile)
        for b in bands_all: # 3) For each band
            col_name = d + '_' + b

            if tile == 0:
                # If the column doesn't exist, create it and populate with 0s
                df_generated[col_name] = 0

            # Load im
            im = load_file(f"data/{t}/{d}/{t[1]}_{b}_{d}.tif")

            # Going four levels deep. Each second on the outside is four weeks in this loop
            # If we die here, there's no waking up.....
            vals = []
            for row, col in df_generated.loc[df_generated.tile == tile][['row_loc', 'col_loc']].values: # 4) For each location of a pixel in a field
                vals.append(im[row][col])
            df_generated.loc[df_generated.tile == tile, col_name] = vals

Tile:  0
20190606
20190701
20190706
20190711
20190721
20190805
20190815
20190825
20190909
20190919
20190924
20191004
20191103
Tile:  1
20190606
20190701
20190706
20190711
20190721
20190805
20190815
20190825
20190909
20190919
20190924
20191004
20191103
Tile:  2
20190606
20190701
20190706
20190711
20190721
20190805
20190815
20190825
20190909
20190919
20190924
20191004
20191103
Tile:  3
20190606
20190701
20190706
20190711
20190721
20190805
20190815
20190825
20190909
20190919
20190924
20191004
20191103


In [15]:
df_generated.head()

,fid,label,row_loc,col_loc,tile,20190606_B01,20190606_B02,20190606_B03,20190606_B04,20190606_B05,...,20191103_B04,20191103_B05,20191103_B06,20191103_B07,20191103_B08,20191103_B8A,20191103_B09,20191103_B11,20191103_B12,20191103_CLD
0,2928,4,214,1278,0,0.1995,0.2024,0.1782,0.1898,0.2462,...,0.0669,0.1239,0.2660,0.3214,0.3204,0.3515,0.361,0.2362,0.1347,0.0
1,2928,4,214,1279,0,0.1995,0.1942,0.1848,0.1846,0.2462,...,0.0603,0.1239,0.2660,0.3214,0.3296,0.3515,0.361,0.2362,0.1347,0.0
2,2928,4,214,1280,0,0.1995,0.1868,0.1930,0.1860,0.2558,...,0.0631,0.1259,0.2579,0.3131,0.3459,0.3307,0.361,0.2304,0.1471,0.0
3,2928,4,214,1281,0,0.1995,0.1962,0.1978,0.1954,0.2558,...,0.1214,0.1259,0.2579,0.3131,0.2861,0.3307,0.361,0.2304,0.1471,0.0
4,2928,4,214,1282,0,0.1995,0.1976,0.1914,0.1956,0.2571,...,0.1278,0.1385,0.2726,0.3239,0.3286,0.3446,0.361,0.2323,0.1355,0.0


In [16]:
df_generated.shape

(67557, 174)

In [ ]:
df_generated.to_csv("data/bands_ungrouped.csv", index = False)

# Import data
> NB: The above "Data Generation" code is preliminary. The data generated isn't used directly, but rather exported to .csv format, and imported below for further usage

In [17]:
sample_submission = pd.read_csv("data/SampleSubmission.csv")

In [18]:
df_ungrouped = pd.read_csv("data/bands_ungrouped.csv")

In [20]:
df_ungrouped.head()

,fid,label,row_loc,col_loc,tile,20190606_B01,20190606_B02,20190606_B03,20190606_B04,20190606_B05,...,20191103_B04,20191103_B05,20191103_B06,20191103_B07,20191103_B08,20191103_B8A,20191103_B09,20191103_B11,20191103_B12,20191103_CLD
0,2928,4,214,1278,0,0.1995,0.2024,0.1782,0.1898,0.2462,...,0.0669,0.1239,0.2660,0.3214,0.3204,0.3515,0.361,0.2362,0.1347,0.0
1,2928,4,214,1279,0,0.1995,0.1942,0.1848,0.1846,0.2462,...,0.0603,0.1239,0.2660,0.3214,0.3296,0.3515,0.361,0.2362,0.1347,0.0
2,2928,4,214,1280,0,0.1995,0.1868,0.1930,0.1860,0.2558,...,0.0631,0.1259,0.2579,0.3131,0.3459,0.3307,0.361,0.2304,0.1471,0.0
3,2928,4,214,1281,0,0.1995,0.1962,0.1978,0.1954,0.2558,...,0.1214,0.1259,0.2579,0.3131,0.2861,0.3307,0.361,0.2304,0.1471,0.0
4,2928,4,214,1282,0,0.1995,0.1976,0.1914,0.1956,0.2571,...,0.1278,0.1385,0.2726,0.3239,0.3286,0.3446,0.361,0.2323,0.1355,0.0


In [21]:
df_ungrouped.shape

(67557, 174)

# DATA PREPARATION/PRE-PROCESSING

In [20]:
# Spatial features to be merged with dataset (by Field ID) later on
row_size = df_ungrouped.groupby("fid")["row_loc"].nunique()
column_size = df_ungrouped.groupby("fid")["col_loc"].nunique()
num_pixels = df_ungrouped.groupby("fid")["label"].count()

In [21]:
# Grouped Data
df_grouped = df_ungrouped.groupby("fid", as_index = False).mean()

### Two separate modelling was done on different set of features, and later combined through ensembling
> ### 1st modelling 
Features used:
1. Pixel values of each of the 12 bands, INCLUDING the cloud probabilities (for the 13 timestamps)
2. Vegetation/Spectral indices like NDVI, GNDVI, AVI etc, and relevant statistics related to the indices like mean, max etc.
3. Spatial features - row_size, column_size (both indicating height, and width), area of field, and number of pixels covered by a field in the area computed  

Relevant code variables are usually suffixed with "_all"  

> ### 2nd modelling  (Pixel related)
Features used:
1. Pixel values of each of the 12 bands, EXCLUDING the cloud probabilities (for the 13 timestamps)
2. Statistics related to pixel values of each band like mean, max etc.  

Relevant code variables are usually suffixed with "_pixels"

In [22]:
# Dataframe for 1st modelling
df_all = df_grouped.copy()

In [23]:
# Dataframe for 2nd modelling
df_pixels = df_grouped.copy()

In [24]:
# Drop non-allowed features. Field ID is dropped later on as it's still needed for further processing
df_all = df_all.drop(columns = ["row_loc", "col_loc", "tile"])
df_pixels = df_pixels.drop(columns = ["row_loc", "col_loc", "tile"])

In [25]:
cloud_columns = ['20190606_CLD', '20190701_CLD', '20190706_CLD', '20190711_CLD', '20190721_CLD', '20190805_CLD', '20190815_CLD', '20190825_CLD', '20190909_CLD', '20190919_CLD', '20190924_CLD', '20191004_CLD', '20191103_CLD']

In [26]:
# Drop Cloud probabilities, and Field ID's in 2nd Dataframe.
# Field ID in 1st Dataframe will be dropped later
df_pixels.drop(columns = cloud_columns + ["fid"], inplace = True)

In [27]:
df_all.head()

,fid,label,20190606_B01,20190606_B02,20190606_B03,20190606_B04,20190606_B05,20190606_B06,20190606_B07,20190606_B08,...,20191103_B04,20191103_B05,20191103_B06,20191103_B07,20191103_B08,20191103_B8A,20191103_B09,20191103_B11,20191103_B12,20191103_CLD
0,1,1.0,0.098600,0.078200,0.108000,0.098600,0.158100,0.309400,0.366800,0.355400,...,0.102800,0.147200,0.263700,0.305800,0.299400,0.320000,0.329700,0.252700,0.153500,0.0
1,2,2.0,0.037855,0.058818,0.086627,0.060945,0.119536,0.275473,0.325918,0.316755,...,0.096155,0.171527,0.323836,0.372173,0.379673,0.410027,0.408255,0.323173,0.207200,0.0
2,3,0.0,0.260700,0.264511,0.232378,0.196911,0.229033,0.301122,0.333367,0.349678,...,0.446889,0.488256,0.522944,0.526311,0.625122,0.521933,0.696400,0.546033,0.435767,100.0
3,4,2.0,0.027792,0.033031,0.060354,0.047331,0.103085,0.260508,0.320369,0.297408,...,0.073946,0.158177,0.341762,0.392846,0.394592,0.419885,0.402800,0.310362,0.200377,0.0
4,5,5.0,0.014600,0.022500,0.035400,0.030200,0.063400,0.154400,0.185300,0.186800,...,0.064100,0.134400,0.286200,0.344500,0.333900,0.357100,0.352600,0.213900,0.126100,0.0


In [28]:
df_pixels.head()

,label,20190606_B01,20190606_B02,20190606_B03,20190606_B04,20190606_B05,20190606_B06,20190606_B07,20190606_B08,20190606_B8A,...,20191103_B03,20191103_B04,20191103_B05,20191103_B06,20191103_B07,20191103_B08,20191103_B8A,20191103_B09,20191103_B11,20191103_B12
0,1.0,0.098600,0.078200,0.108000,0.098600,0.158100,0.309400,0.366800,0.355400,0.391000,...,0.104200,0.102800,0.147200,0.263700,0.305800,0.299400,0.320000,0.329700,0.252700,0.153500
1,2.0,0.037855,0.058818,0.086627,0.060945,0.119536,0.275473,0.325918,0.316755,0.348482,...,0.109055,0.096155,0.171527,0.323836,0.372173,0.379673,0.410027,0.408255,0.323173,0.207200
2,0.0,0.260700,0.264511,0.232378,0.196911,0.229033,0.301122,0.333367,0.349678,0.340689,...,0.510644,0.446889,0.488256,0.522944,0.526311,0.625122,0.521933,0.696400,0.546033,0.435767
3,2.0,0.027792,0.033031,0.060354,0.047331,0.103085,0.260508,0.320369,0.297408,0.338000,...,0.093308,0.073946,0.158177,0.341762,0.392846,0.394592,0.419885,0.402800,0.310362,0.200377
4,5.0,0.014600,0.022500,0.035400,0.030200,0.063400,0.154400,0.185300,0.186800,0.187600,...,0.088300,0.064100,0.134400,0.286200,0.344500,0.333900,0.357100,0.352600,0.213900,0.126100


### Vegetation/Spectral indices

1. Normalized Difference Vegetation Index (NDVI)
2. Green Normalized Difference Vegetation Index (GNDVI)
3. Enhanced Vegetation Index (EVI)
4. Enhanced Vegetation Index 2 (EVI2)
5. Advanced Vegetation Index (AVI)
6. Bare Soil Index (BSI)
7. Shadow Index (SI)
8. Normalized Difference Water Index (NDWI)
9. Normalized Difference Moisture Index (NDMI)
10. Normalized Pigment Chlorophyll Ratio Index (NPCRI)

In [29]:
spectral_indices = ["NDVI", "GNDVI", "EVI", "EVI2", "AVI", "BSI", "SI", "NDWI", "NDMI", "NPCRI"]

In [30]:
for i in range(13):
#     Band Pixel values per timestamp
    b1 = df_all.filter(like = "B01").values[:,i]
    b2 = df_all.filter(like = "B02").values[:,i]
    b3 = df_all.filter(like = "B03").values[:,i]
    b4 = df_all.filter(like = "B04").values[:,i]
    b5 = df_all.filter(like = "B05").values[:,i]
    b6 = df_all.filter(like = "B06").values[:,i]
    b7 = df_all.filter(like = "B07").values[:,i]
    b8 = df_all.filter(like = "B08").values[:,i]
    b8a = df_all.filter(like = "B8A").values[:,i]
    b9 = df_all.filter(like = "B09").values[:,i]    
    b11 = df_all.filter(like = "B11").values[:,i]
    b12 = df_all.filter(like = "B12").values[:,i]
    
#     Computation of indices
    ndvi = (b8 - b4) / (b8 + b4)
    gndvi = (b8 - b3) / (b8 + b3)
    evi = 2.5 * (b8 - b4) / ((b8 + 6.0 * b4 - 7.5 * b2) + 1.0)    
    evi2 = 2.4 * (b8 - b4) / (b8 + b4 + 1.0)
    avi = (b8 * (1 - b4) * (b8 - b4))
    bsi = ((b11 + b4) - (b8 + b2)) / ((b11 + b4) + (b8 + b2))
    si = ((1 - b2) * (1 - b3) * (1 - b4))
    ndwi = (b3 - b8) / (b3 + b8)
    ndmi = (b8 - b11) / (b8 + b11)
    npcri = (b4 - b2) / (b4 + b2) 
    
#     Add indices as features to 1st dataframe per timestamp
    df_all[f'NDVI_{dates[i]}'] = ndvi 
    df_all[f'GNDVI_{dates[i]}'] = gndvi
    df_all[f'EVI_{dates[i]}'] = evi
    df_all[f'EVI2_{dates[i]}'] = evi2
    df_all[f'AVI_{dates[i]}'] = avi
    df_all[f'BSI_{dates[i]}'] = bsi
    df_all[f'SI_{dates[i]}'] = si    
    df_all[f'NDWI_{dates[i]}'] = ndwi
    df_all[f'NDMI_{dates[i]}'] = ndmi
    df_all[f'NPCRI_{dates[i]}'] = npcri

In [31]:
# Add spectral indices statistics related to 1st dataframe
for i in spectral_indices:
    df_all[f'{i}_min'] = df_all.filter(regex = f'^{i}').min(axis = 1)
    df_all[f'{i}_max'] = df_all.filter(regex = f'^{i}').max(axis = 1)
    df_all[f'{i}_avg'] = df_all.filter(regex = f'^{i}').mean(axis = 1)
    df_all[f'{i}_std'] = df_all.filter(regex = f'^{i}').std(axis = 1)    

In [32]:
# Add band pixel values statistics related to 2nd dataframe
for i in bands:
    df_pixels[f'{i}_std'] = df_pixels.filter(like = f'_{i}').std(axis = 1)
    df_pixels[f'{i}_max'] = df_pixels.filter(like = f'_{i}').max(axis = 1)
    df_pixels[f'{i}_min'] = df_pixels.filter(like = f'_{i}').min(axis = 1)
    df_pixels[f'{i}_avg'] = df_pixels.filter(like = f'_{i}').mean(axis = 1)

In [33]:
# NB: "new_df" represents 1st dataframe
new_df = df_all.copy()

## Spatial statistics - used in first dataframe

In [34]:
new_df["row_size"] = new_df.fid.map(row_size)
new_df["col_size"] = new_df.fid.map(column_size)
new_df["area"] = new_df.apply(lambda row: row.row_size * row.col_size, axis = 1)
# number of pixels covered by a field in the area computed
new_df["num_pixels"] = new_df.fid.map(num_pixels)

In [35]:
# Drop non-allowed Field ID in 1st dataframe
new_df = new_df.drop(columns = ["fid"])

## Train, and Test set generation

In [36]:
# 1st dataframe
df_all_train = new_df[new_df.label != 0].copy()
df_all_test = new_df[new_df.label == 0].copy()

df_all_train = df_all_train.reset_index(drop = True)
df_all_test = df_all_test.reset_index(drop = True)

In [37]:
# 2nd dataframe
df_pixels_train = df_pixels[df_pixels.label != 0].copy()
df_pixels_test = df_pixels[df_pixels.label == 0].copy()

df_pixels_train = df_pixels_train.reset_index(drop = True)
df_pixels_test = df_pixels_test.reset_index(drop = True)

In [38]:
(df_all_train.shape, df_all_test.shape), (df_pixels_train.shape, df_pixels_test.shape)

(((3286, 344), (1402, 344)), ((3286, 205), (1402, 205)))

In [39]:
# Drop crop labels (0) from test sets
df_all_test.drop("label", inplace = True, axis = 1)
df_pixels_test.drop("label", inplace = True, axis = 1)

## Train X, and y generation (Features, and Target/Label)

In [40]:
# 1st dataframe
train_X_all = df_all_train.drop("label", axis = 1)
train_y_all = df_all_train.label

In [41]:
# 2nd dataframe
train_X_pixels = df_pixels_train.drop("label", axis = 1)
train_y_pixels = df_pixels_train.label

In [42]:
train_X_all.shape, df_all_test.shape

((3286, 343), (1402, 343))

In [43]:
train_X_pixels.shape, df_pixels_test.shape

((3286, 204), (1402, 204))

## Class/Label weights

In [44]:
# Class weights for cross validation
# Cross validation weights are based on 80% of train set
X_trn, X_val, y_trn, y_val = train_test_split(train_X_all, train_y_all, test_size = 0.2, stratify = train_y_all, random_state = 5, shuffle = True)
label_weights1 = compute_class_weight("balanced", np.unique(y_trn), y_trn)

In [45]:
# Class weights for full training
# Cross validation weights are based on full train set
label_weights2 = compute_class_weight("balanced", np.unique(train_y_all), train_y_all)

## Permutation Importance for Feature Selection
> NB: Feature selection/dropping was only carried out on 1st dataframe

In [46]:
# cb_pi --> catboost_permutation_importance
cb_pi = CatBoostClassifier(n_estimators = 1400, learning_rate = 0.03, random_state = 11, task_type = "GPU")
cb_pi.fit(X_trn, y_trn)

0:	learn: 1.9110878	total: 32.7ms	remaining: 45.8s
1:	learn: 1.8784226	total: 46.5ms	remaining: 32.5s
2:	learn: 1.8497764	total: 64.7ms	remaining: 30.1s
3:	learn: 1.8228234	total: 78.2ms	remaining: 27.3s
4:	learn: 1.7973512	total: 91.6ms	remaining: 25.5s
5:	learn: 1.7730135	total: 105ms	remaining: 24.3s
6:	learn: 1.7504011	total: 118ms	remaining: 23.5s
7:	learn: 1.7282823	total: 132ms	remaining: 22.9s
8:	learn: 1.7073400	total: 145ms	remaining: 22.4s
9:	learn: 1.6875321	total: 159ms	remaining: 22s
10:	learn: 1.6692760	total: 172ms	remaining: 21.7s
11:	learn: 1.6521108	total: 185ms	remaining: 21.4s
12:	learn: 1.6359421	total: 198ms	remaining: 21.2s
13:	learn: 1.6208647	total: 212ms	remaining: 20.9s
14:	learn: 1.6064373	total: 230ms	remaining: 21.2s
15:	learn: 1.5917729	total: 243ms	remaining: 21s
16:	learn: 1.5786597	total: 256ms	remaining: 20.8s
17:	learn: 1.5654139	total: 268ms	remaining: 20.6s
18:	learn: 1.5529085	total: 281ms	remaining: 20.5s
19:	learn: 1.5407589	total: 294ms	remain

172:	learn: 1.0676485	total: 2.31s	remaining: 16.4s
173:	learn: 1.0661548	total: 2.33s	remaining: 16.4s
174:	learn: 1.0644732	total: 2.34s	remaining: 16.4s
175:	learn: 1.0632347	total: 2.35s	remaining: 16.4s
176:	learn: 1.0623289	total: 2.37s	remaining: 16.4s
177:	learn: 1.0607949	total: 2.38s	remaining: 16.4s
178:	learn: 1.0599336	total: 2.4s	remaining: 16.3s
179:	learn: 1.0584726	total: 2.41s	remaining: 16.3s
180:	learn: 1.0569578	total: 2.42s	remaining: 16.3s
181:	learn: 1.0557260	total: 2.43s	remaining: 16.3s
182:	learn: 1.0547360	total: 2.45s	remaining: 16.3s
183:	learn: 1.0535540	total: 2.46s	remaining: 16.3s
184:	learn: 1.0520574	total: 2.47s	remaining: 16.2s
185:	learn: 1.0511166	total: 2.48s	remaining: 16.2s
186:	learn: 1.0496436	total: 2.5s	remaining: 16.2s
187:	learn: 1.0478244	total: 2.51s	remaining: 16.2s
188:	learn: 1.0465361	total: 2.53s	remaining: 16.2s
189:	learn: 1.0450755	total: 2.54s	remaining: 16.2s
190:	learn: 1.0440312	total: 2.55s	remaining: 16.1s
191:	learn: 1.

344:	learn: 0.8703229	total: 4.6s	remaining: 14.1s
345:	learn: 0.8690641	total: 4.62s	remaining: 14.1s
346:	learn: 0.8680304	total: 4.63s	remaining: 14.1s
347:	learn: 0.8673412	total: 4.64s	remaining: 14s
348:	learn: 0.8657397	total: 4.66s	remaining: 14s
349:	learn: 0.8647964	total: 4.67s	remaining: 14s
350:	learn: 0.8639954	total: 4.68s	remaining: 14s
351:	learn: 0.8626991	total: 4.7s	remaining: 14s
352:	learn: 0.8614793	total: 4.71s	remaining: 14s
353:	learn: 0.8606206	total: 4.72s	remaining: 14s
354:	learn: 0.8594217	total: 4.74s	remaining: 13.9s
355:	learn: 0.8587028	total: 4.75s	remaining: 13.9s
356:	learn: 0.8579901	total: 4.76s	remaining: 13.9s
357:	learn: 0.8571575	total: 4.78s	remaining: 13.9s
358:	learn: 0.8559198	total: 4.79s	remaining: 13.9s
359:	learn: 0.8547084	total: 4.81s	remaining: 13.9s
360:	learn: 0.8536742	total: 4.82s	remaining: 13.9s
361:	learn: 0.8524911	total: 4.83s	remaining: 13.9s
362:	learn: 0.8518884	total: 4.85s	remaining: 13.8s
363:	learn: 0.8508147	total:

506:	learn: 0.7271053	total: 6.86s	remaining: 12.1s
507:	learn: 0.7259196	total: 6.88s	remaining: 12.1s
508:	learn: 0.7249172	total: 6.89s	remaining: 12.1s
509:	learn: 0.7243086	total: 6.9s	remaining: 12s
510:	learn: 0.7232543	total: 6.92s	remaining: 12s
511:	learn: 0.7226652	total: 6.95s	remaining: 12.1s
512:	learn: 0.7218253	total: 6.97s	remaining: 12.1s
513:	learn: 0.7211689	total: 6.99s	remaining: 12s
514:	learn: 0.7204164	total: 7s	remaining: 12s
515:	learn: 0.7196692	total: 7.01s	remaining: 12s
516:	learn: 0.7186639	total: 7.03s	remaining: 12s
517:	learn: 0.7181281	total: 7.04s	remaining: 12s
518:	learn: 0.7174093	total: 7.05s	remaining: 12s
519:	learn: 0.7164509	total: 7.07s	remaining: 12s
520:	learn: 0.7158168	total: 7.08s	remaining: 11.9s
521:	learn: 0.7154142	total: 7.09s	remaining: 11.9s
522:	learn: 0.7147772	total: 7.11s	remaining: 11.9s
523:	learn: 0.7139057	total: 7.12s	remaining: 11.9s
524:	learn: 0.7133364	total: 7.13s	remaining: 11.9s
525:	learn: 0.7126488	total: 7.14s

679:	learn: 0.6114886	total: 9.13s	remaining: 9.67s
680:	learn: 0.6106690	total: 9.14s	remaining: 9.65s
681:	learn: 0.6102421	total: 9.15s	remaining: 9.64s
682:	learn: 0.6096223	total: 9.17s	remaining: 9.63s
683:	learn: 0.6091141	total: 9.18s	remaining: 9.61s
684:	learn: 0.6085849	total: 9.19s	remaining: 9.6s
685:	learn: 0.6077979	total: 9.2s	remaining: 9.58s
686:	learn: 0.6072251	total: 9.22s	remaining: 9.56s
687:	learn: 0.6065114	total: 9.23s	remaining: 9.55s
688:	learn: 0.6059073	total: 9.24s	remaining: 9.54s
689:	learn: 0.6055135	total: 9.25s	remaining: 9.52s
690:	learn: 0.6048971	total: 9.27s	remaining: 9.51s
691:	learn: 0.6045254	total: 9.28s	remaining: 9.49s
692:	learn: 0.6040037	total: 9.29s	remaining: 9.48s
693:	learn: 0.6035204	total: 9.3s	remaining: 9.46s
694:	learn: 0.6029051	total: 9.31s	remaining: 9.45s
695:	learn: 0.6020133	total: 9.33s	remaining: 9.44s
696:	learn: 0.6015693	total: 9.34s	remaining: 9.42s
697:	learn: 0.6010659	total: 9.36s	remaining: 9.41s
698:	learn: 0.6

838:	learn: 0.5211024	total: 11.3s	remaining: 7.54s
839:	learn: 0.5204546	total: 11.3s	remaining: 7.55s
840:	learn: 0.5199811	total: 11.4s	remaining: 7.55s
841:	learn: 0.5192579	total: 11.4s	remaining: 7.54s
842:	learn: 0.5187997	total: 11.4s	remaining: 7.54s
843:	learn: 0.5185178	total: 11.4s	remaining: 7.53s
844:	learn: 0.5181731	total: 11.5s	remaining: 7.53s
845:	learn: 0.5176493	total: 11.5s	remaining: 7.52s
846:	learn: 0.5171570	total: 11.5s	remaining: 7.51s
847:	learn: 0.5167023	total: 11.5s	remaining: 7.51s
848:	learn: 0.5160000	total: 11.6s	remaining: 7.5s
849:	learn: 0.5155476	total: 11.6s	remaining: 7.49s
850:	learn: 0.5150963	total: 11.6s	remaining: 7.49s
851:	learn: 0.5148565	total: 11.6s	remaining: 7.48s
852:	learn: 0.5142564	total: 11.7s	remaining: 7.47s
853:	learn: 0.5137547	total: 11.7s	remaining: 7.47s
854:	learn: 0.5133524	total: 11.7s	remaining: 7.46s
855:	learn: 0.5129604	total: 11.7s	remaining: 7.46s
856:	learn: 0.5120622	total: 11.8s	remaining: 7.45s
857:	learn: 0

1000:	learn: 0.4474084	total: 13.8s	remaining: 5.5s
1001:	learn: 0.4471264	total: 13.8s	remaining: 5.49s
1002:	learn: 0.4467043	total: 13.8s	remaining: 5.47s
1003:	learn: 0.4462454	total: 13.8s	remaining: 5.46s
1004:	learn: 0.4458689	total: 13.8s	remaining: 5.44s
1005:	learn: 0.4455180	total: 13.9s	remaining: 5.43s
1006:	learn: 0.4451869	total: 13.9s	remaining: 5.42s
1007:	learn: 0.4449006	total: 13.9s	remaining: 5.4s
1008:	learn: 0.4444851	total: 13.9s	remaining: 5.39s
1009:	learn: 0.4441222	total: 13.9s	remaining: 5.38s
1010:	learn: 0.4435705	total: 13.9s	remaining: 5.36s
1011:	learn: 0.4433624	total: 13.9s	remaining: 5.35s
1012:	learn: 0.4429922	total: 14s	remaining: 5.33s
1013:	learn: 0.4425122	total: 14s	remaining: 5.32s
1014:	learn: 0.4420512	total: 14s	remaining: 5.3s
1015:	learn: 0.4415586	total: 14s	remaining: 5.29s
1016:	learn: 0.4410788	total: 14s	remaining: 5.28s
1017:	learn: 0.4407061	total: 14s	remaining: 5.26s
1018:	learn: 0.4404438	total: 14s	remaining: 5.25s
1019:	lear

1157:	learn: 0.3902846	total: 15.9s	remaining: 3.31s
1158:	learn: 0.3898085	total: 15.9s	remaining: 3.3s
1159:	learn: 0.3894291	total: 15.9s	remaining: 3.29s
1160:	learn: 0.3889404	total: 15.9s	remaining: 3.27s
1161:	learn: 0.3885404	total: 15.9s	remaining: 3.26s
1162:	learn: 0.3881182	total: 15.9s	remaining: 3.25s
1163:	learn: 0.3878711	total: 15.9s	remaining: 3.23s
1164:	learn: 0.3873562	total: 15.9s	remaining: 3.22s
1165:	learn: 0.3870813	total: 16s	remaining: 3.2s
1166:	learn: 0.3867158	total: 16s	remaining: 3.2s
1167:	learn: 0.3865568	total: 16.1s	remaining: 3.2s
1168:	learn: 0.3861862	total: 16.2s	remaining: 3.19s
1169:	learn: 0.3859566	total: 16.2s	remaining: 3.19s
1170:	learn: 0.3856234	total: 16.3s	remaining: 3.18s
1171:	learn: 0.3852951	total: 16.3s	remaining: 3.17s
1172:	learn: 0.3849184	total: 16.3s	remaining: 3.16s
1173:	learn: 0.3845635	total: 16.3s	remaining: 3.15s
1174:	learn: 0.3841636	total: 16.4s	remaining: 3.13s
1175:	learn: 0.3840253	total: 16.4s	remaining: 3.12s
1

1320:	learn: 0.3373578	total: 18.9s	remaining: 1.13s
1321:	learn: 0.3371779	total: 18.9s	remaining: 1.11s
1322:	learn: 0.3368335	total: 18.9s	remaining: 1.1s
1323:	learn: 0.3365342	total: 18.9s	remaining: 1.08s
1324:	learn: 0.3361407	total: 18.9s	remaining: 1.07s
1325:	learn: 0.3358863	total: 18.9s	remaining: 1.06s
1326:	learn: 0.3356141	total: 18.9s	remaining: 1.04s
1327:	learn: 0.3354472	total: 19s	remaining: 1.03s
1328:	learn: 0.3351706	total: 19s	remaining: 1.01s
1329:	learn: 0.3349847	total: 19s	remaining: 999ms
1330:	learn: 0.3347353	total: 19s	remaining: 984ms
1331:	learn: 0.3345816	total: 19s	remaining: 970ms
1332:	learn: 0.3341688	total: 19s	remaining: 956ms
1333:	learn: 0.3339655	total: 19s	remaining: 941ms
1334:	learn: 0.3337345	total: 19s	remaining: 927ms
1335:	learn: 0.3335595	total: 19s	remaining: 913ms
1336:	learn: 0.3333898	total: 19.1s	remaining: 898ms
1337:	learn: 0.3331787	total: 19.1s	remaining: 884ms
1338:	learn: 0.3328876	total: 19.1s	remaining: 870ms
1339:	learn:

In [47]:
pi = PermutationImportance(cb_pi, random_state = 90, n_iter = 5)
pi.fit(X_val, y_val)

PermutationImportance(cv='prefit',
                      estimator=<catboost.core.CatBoostClassifier object at 0x7fe2cdd7d940>,
                      n_iter=5, random_state=90, refit=True, scoring=None)

In [48]:
eli5.show_weights(pi, feature_names = train_X_all.columns.tolist(), top = None)

Weight,Feature
0.0103 ± 0.0059,20191004_B05
0.0088 ± 0.0045,NPCRI_20190701
0.0082 ± 0.0081,20190825_B09
0.0082 ± 0.0024,AVI_20190701
0.0079 ± 0.0040,area
0.0079 ± 0.0030,20190606_B02
0.0076 ± 0.0019,AVI_20190711
0.0073 ± 0.0052,num_pixels
0.0073 ± 0.0056,NDVI_std
0.0067 ± 0.0036,NPCRI_20190805


In [49]:
pi_results = eli5.formatters.as_dataframe.explain_weights_df(pi, feature_names = train_X_all.columns.tolist())

In [50]:
# feature importance weigth threshold is 0
low_importance = pi_results[pi_results.weight <= 0].feature.values

In [53]:
low_importance

numpy.ndarray

## Drop below features based on Permutation Importance (PI) performed above
> NB: Features returned by the PI above may not be the same as the features I decided to eventually drop due to subjective reasons

In [ ]:
features_to_drop = [
    '20190815_CLD', '20191004_CLD', '20190919_CLD', '20190701_CLD',
    '20190924_CLD', '20190706_CLD', 'SI_avg', '20190606_B07',
    'BSI_max', 'BSI_20191103', '20190805_B04', '20191004_B03',
    'SI_std', 'NPCRI_20190606', '20190919_B03', '20190805_B08',
    'NPCRI_20190909', 'EVI_min', '20190721_B07', '20190711_B07',
    '20190721_B05', 'AVI_20191103', 'SI_20190924', '20190805_B09',
    '20190706_B09', 'EVI_20191004', 'EVI_20190711', '20191004_B08',
    'NDVI_20191004', '20190825_B8A', '20190919_B01', '20190805_B06',
    'GNDVI_20190706', 'SI_20190805', 'EVI2_20190805', '20190706_B11',
    'col_size', '20190701_B04', 'GNDVI_20191004', 'GNDVI_20190924',
    'NPCRI_max', 'EVI_20190701', 'BSI_20190711', '20190805_B12',
    'SI_20191103', '20190919_B04', 'NDMI_20190924', 'BSI_avg',
    'NPCRI_20190706', '20190919_B8A', '20190909_B05', '20190706_B06',
    'GNDVI_20190606', '20190924_B8A', 'NDWI_20190701', 'AVI_avg',
    'GNDVI_20190919', 'NDVI_20190606', 'NDVI_20191103', 'SI_20190919',
    'GNDVI_20190721', 'NPCRI_20190721', '20190919_B09', '20190606_B01',
    '20190825_B05', '20190909_B09', '20191004_B01', '20190805_B03',
    'GNDVI_20191103', 'NPCRI_20190919', 'EVI_20190805', '20190706_B07',
    '20190909_CLD', '20190825_B12', '20190606_B06', '20190909_B07',
    'BSI_20190825', '20190805_B8A', '20190924_B09', '20190701_B07',
    'EVI2_20190721', 'NPCRI_20190805'
]

In [ ]:
train_X_all = train_X_all.drop(columns = features_to_drop)
df_all_test = df_all_test.drop(columns = features_to_drop)

## MODELLING

cb = Catboost model - Dataframe 1  
cb2 = Catboost (with class weights) -  Dataframe 1  

cb_pixels = Catboost -  Dataframe 2  
cb2_pixels = Catboost (with class weights) -  Dataframe 2 

lda = LinearDiscriminantAnalysis model  
bc = BaggingClassifier (Bagged Ensemble) with lda as it's base estimator  

**NB: Catboost is trained with GPU enabled**

In [ ]:
cb = CatBoostClassifier(n_estimators = 1500, learning_rate=0.03, depth = 6, random_state = 11, bagging_temperature = 1, task_type = "GPU")
# Use "class_weights = label_weights1" for cross validation
cb2 = CatBoostClassifier(n_estimators = 1100, learning_rate=0.03, depth = 6, random_state = 11, bagging_temperature = 1, task_type = "GPU", class_weights = label_weights2)

cb_pixels = CatBoostClassifier(n_estimators = 1500, learning_rate=0.03, depth = 6, random_state = 11, bagging_temperature = 1, task_type = "GPU")
# Use "class_weights = label_weights1" for cross validation
cb2_pixels = CatBoostClassifier(n_estimators = 1100, learning_rate=0.03, depth = 6, random_state = 11, bagging_temperature = 1, task_type = "GPU", class_weights = label_weights2)

lda = LinearDiscriminantAnalysis()
bc = BaggingClassifier(base_estimator = lda, n_estimators = 30, random_state = 0)

## Cross Validation (CV)
Cross validation is also used to determine weights to be applied to the average (weighted averga) of models ensembled later on.

## CV with 1st DataFrame

In [ ]:
# Catboost
cv_all_1 = cross_val_predict(cb, train_X_all, train_y_all, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Catboost with weights
cv_all_2 = cross_val_predict(cb2, train_X_all, train_y_all, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Bagged LDA
cv_all_3 = cross_val_predict(bc, train_X_all, train_y_all, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Weighted Average of above 3 (in two steps)
cv_all_1_2 = (0.72 * cv_all_1) + ((1 - 0.72) * cv_all_2)
cv_all_1_2_3 = (0.7 * cv_all_1_2) + ((1 - 0.7) * cv_all_3)

## CV with 2nd DataFrame

In [ ]:
# Catboost
cv_pixels_1 = cross_val_predict(cb_pixels, train_X_pixels, train_y_pixels, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Catboost with weights
cv_pixels_2 = cross_val_predict(cb2_pixels, train_X_pixels, train_y_pixels, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Bagged LDA
cv_pixels_3 = cross_val_predict(bc, train_X_pixels, train_y_pixels, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Weighted Average of above 3 (in two steps)
cv_pixels_1_2 = (0.76 * cv_pixels_1) + ((1 - 0.76) * cv_pixels_2)
cv_pixels_1_2_3 = (0.67 * cv_pixels_1_2 ) + ((1 - 0.67) * cv_pixels_3)

In [ ]:
# Code to determine weights
scores = []

for w in range(0, 100):
    w = w / 100.
#     two cross-validation results of choice should be imputed to determine appropriate weights
    scores.append(log_loss(train_y_all, (w * cv_all_1_2_3) + ((1 - w) * cv_pixels_1_2_3)))

best_score = min(scores)
weight = scores.index(best_score) / 100.

print(weight, best_score)

# TRAINING

## Modelling Architecture - Explanation

The two different datasets each have the same model pipeline.  

For each of the datasets:  
### Level 1
Training is done using both a CatboostClassifier (without class_weights), and another CatboostClassifier (with class_weights), denoted by "cb" and "cb2" respectively.  
The weighted average of the predictions from both classifiers are obtained, using:   
1. 72% of cb,  and 28% of cb2 for dataset 1;
2. 76% of cb, and 24% of cb2 for dataset 2  

### Level 2
The model used for training here is a bagged ensemble (using sklearn's BaggingClassifier), with the LinearDiscriminantAnalysis alogrithm as the base estimator.  
The predicitons from the bagged classifier is then averaged (weighted) with the individual results from level 1, using:   
1. 70% of level 1, and 30% of bagged classifier for dataset 1;  
2. 67% of level 1, and 33% of bagged classifier for dataset 2  

### Level 3
This level simply finds the weighted average of the final predictions from both datasets.  
It takes 68% of the final predictions from dataset 1, and 32% from dataset 2.  

## 1st part of model - using dataset 1

In [ ]:
cb.fit(train_X_all, train_y_all)

In [ ]:
cb2.fit(train_X_all, train_y_all)

In [ ]:
bc.fit(train_X_all, train_y_all)

### Predictions

In [ ]:
test_preds_all_1 = cb.predict_proba(df_all_test)
test_preds_all_2 = cb2.predict_proba(df_all_test)
test_preds_all_3 = bc.predict_proba(df_all_test)

### Weighted Average

In [ ]:
# Level 1
test_preds_all = (0.72 * test_preds_all_1) + ((1 - 0.72) * test_preds_all_2)
# Level 2
test_preds_all = (0.7 * test_preds_all) + ((1 - 0.7) * test_preds_all_3)

## 2nd part of model - using dataset 2

In [ ]:
cb_pixels.fit(train_X_pixels, train_y_pixels)

In [ ]:
cb2_pixels.fit(train_X_pixels, train_y_pixels)

In [ ]:
bc.fit(train_X_pixels, train_y_pixels)

### Predictions

In [ ]:
test_preds_pixels_1 = cb_pixels.predict_proba(df_pixels_test)
test_preds_pixels_2 = cb2_pixels.predict_proba(df_pixels_test)
test_preds_pixels_3 = bc.predict_proba(df_pixels_test)

### Weighted Average

In [ ]:
# Level 1
test_preds_pixels = (0.76 * test_preds_pixels_1) + ((1 - 0.76) * test_preds_pixels_2)
# Level 2
test_preds_pixels = (0.67 * test_preds_pixels) + ((1 - 0.67) * test_preds_pixels_3)

## Final Predictions - Weighted Average of 2 parts

In [ ]:
# Level 3
test_preds = (0.68 * test_preds_all) + ((1 - 0.68) * test_preds_pixels)

### Convert to DataFrame

In [ ]:
test_preds = pd.DataFrame(test_preds)

### Merge predictions with Sample Submission format

In [ ]:
sample_submission.Crop_ID_1 = test_preds[0]
sample_submission.Crop_ID_2 = test_preds[1]
sample_submission.Crop_ID_3 = test_preds[2]
sample_submission.Crop_ID_4 = test_preds[3]
sample_submission.Crop_ID_5 = test_preds[4]
sample_submission.Crop_ID_6 = test_preds[5]
sample_submission.Crop_ID_7 = test_preds[6]

In [ ]:
sample_submission.head()

### Export 

In [ ]:
sample_submission.to_csv("SampleSub.csv", index = False)